# Pre-Processing Dataset

# Load and Clean Dataset

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Libraries

In [19]:
import time
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, label_binarize
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# Define Features of Interest

In [20]:
req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Min',' Fwd Packet Length Mean',' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std','Flow Bytes/s',' Flow Packets/s',' Flow IAT Mean',' Flow IAT Std',' Flow IAT Max',' Flow IAT Min','Fwd IAT Total',' Fwd IAT Mean',' Fwd IAT Std',' Fwd IAT Max',' Fwd IAT Min','Bwd IAT Total',' Bwd IAT Mean',' Bwd IAT Std',' Bwd IAT Max',' Bwd IAT Min','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags',' Fwd Header Length',' Bwd Header Length','Fwd Packets/s',' Bwd Packets/s',' Min Packet Length',' Max Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',' ACK Flag Count',' URG Flag Count',' CWE Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets',' Subflow Fwd Bytes',' Subflow Bwd Packets',' Subflow Bwd Bytes','Init_Win_bytes_forward',' Init_Win_bytes_backward',' act_data_pkt_fwd',' min_seg_size_forward','Active Mean',' Active Std',' Active Max',' Active Min','Idle Mean',' Idle Std',' Idle Max',' Idle Min',' Label']

# Load and Inspect the Dataset

In [21]:
fraction = 1

df0 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

df1 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df2 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', encoding='ISO-8859-1', usecols=req_cols).sample(frac = fraction)


df3 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df4 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df5 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df6 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df7 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


frames = [df0, df1, df2, df3, df4, df5,df6, df7]

df = pd.concat(frames,ignore_index=True)

df = df.sample(frac=1)  # Shuffle dataset

df.columns = df.columns.str.strip()


<ipython-input-21-fdef386a192d>:8: DtypeWarning: Columns (84) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/Datasets/CIC-IDS-2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', encoding='ISO-8859-1', usecols=req_cols).sample(frac = fraction)


# Fix Corrupt Labels

In [22]:
# **🔹 Fix Corrupted Labels (Web Attack Variants)**
label_mapping = {
    "Web Attack � Sql Injection": "Web Attack",
    "Web Attack � Brute Force": "Web Attack",
    "Web Attack � XSS": "Web Attack",
    "Web Attack   Sql Injection": "Web Attack",
    "Web Attack   Brute Force": "Web Attack",
    "Web Attack   XSS": "Web Attack",
    "Web Attack - Sql Injection": "Web Attack",
    "Web Attack - Brute Force": "Web Attack",
    "Web Attack - XSS": "Web Attack",
    "Web Attack \x96 Sql Injection": "Web Attack",
    "Web Attack \x96 Brute Force": "Web Attack",
    "Web Attack \x96 XSS": "Web Attack"
}

df["Label"] = df["Label"].replace(label_mapping)
print("Unique labels in the dataset after cleaning:")
print(df["Label"].unique())

Unique labels in the dataset after cleaning:
['DoS Hulk' 'BENIGN' nan 'PortScan' 'DoS Slowhttptest' 'DDoS'
 'SSH-Patator' 'FTP-Patator' 'DoS GoldenEye' 'DoS slowloris' 'Web Attack'
 'Bot' 'Infiltration' 'Heartbleed']


In [23]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
378493,80.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS Hulk
1897710,53.0,89650.0,1.0,1.0,42.0,74.0,42.0,42.0,42.000000,0.000000,...,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2795450,80.0,9381332.0,86.0,95.0,3412.0,128512.0,433.0,0.0,39.674419,124.643628,...,32.0,3117366.0,0.0,3117366.0,3117366.0,6176316.0,0.0,6176316.0,6176316.0,BENIGN
2404801,443.0,4845208.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1097850,49716.0,2.0,2.0,0.0,37.0,0.0,37.0,0.0,18.500000,26.162951,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


# Data Normalization

In [24]:
df_max_scaled = df.copy()

# Remove label temporarily for normalization
df_max_scaled.pop('Label')

# Apply Min-Max Scaling (Feature Normalization)
for col in df_max_scaled.columns:
    max_val = abs(df_max_scaled[col].max())
    if max_val > 0:  # Avoid division by zero
        df_max_scaled[col] = df_max_scaled[col] / max_val

# Reassign the label column after scaling
df = df_max_scaled.assign(Label=df["Label"])

# Fill NaN values (if any)
df = df.fillna(0)

print("Dataset normalization completed.")


# # ✅ NORMALIZE NUMERIC FEATURES
# features = [col for col in df.columns if col != "Label"]
# scaler = MinMaxScaler()
# df[features] = scaler.fit_transform(df[features])
# df = df.fillna(0)  # Fill any remaining NaN values

Dataset normalization completed.


In [25]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
378493,0.001221,8.333333e-09,0.000009,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.231884,0.00000,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,DoS Hulk
1897710,0.000809,7.470833e-04,0.000005,0.000003,3.255814e-06,1.128990e-07,0.001692,0.018065,0.007070,0.000000,...,0.289855,0.00000,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,BENIGN
2795450,0.001221,7.817777e-02,0.000391,0.000325,2.644961e-04,1.960659e-04,0.017446,0.000000,0.006678,0.017492,...,0.231884,0.02834,0.0,0.02834,0.02834,0.051469,0.0,0.051469,0.051469,BENIGN
2404801,0.006760,4.037673e-02,0.000009,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.144928,0.00000,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,BENIGN
1097850,0.758618,1.666667e-08,0.000009,0.000000,2.868217e-06,0.000000e+00,0.001491,0.000000,0.003114,0.003672,...,0.231884,0.00000,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,BENIGN


# Count and Remove Duplicates

In [26]:
# # Separate labels and features
# y = df.pop('Label')
# X = df

# # Print label distribution
# counter = Counter(y)
# print("Initial class distribution:", counter)

# # Assign features back and remove duplicates
# df = X.assign(Label=y)
# df = df.drop_duplicates()

# # Check distribution after removing duplicates
# y = df.pop('Label')
# X = df
# counter = Counter(y)
# print("After removing duplicates:", counter)

# # Assign features and labels again
# df = X.assign(Label=y)

# ✅ CHECK LABEL DISTRIBUTION BEFORE REMOVAL
print("Class distribution before duplicate removal:\n", Counter(df["Label"]))

# ✅ REMOVE DUPLICATE ROWS
df = df.drop_duplicates()

Class distribution before duplicate removal:
 Counter({'BENIGN': 2273097, 0: 288602, 'DoS Hulk': 231073, 'PortScan': 158930, 'DDoS': 128027, 'DoS GoldenEye': 10293, 'FTP-Patator': 7938, 'SSH-Patator': 5897, 'DoS slowloris': 5796, 'DoS Slowhttptest': 5499, 'Web Attack': 2180, 'Bot': 1966, 'Infiltration': 36, 'Heartbleed': 11})


# Encode Labels

In [27]:
import joblib
from sklearn.preprocessing import LabelEncoder

# ✅ PRINT UNIQUE LABELS BEFORE ENCODING
print("Unique labels before encoding:", df["Label"].unique())
# ✅ FIX LABELS BEFORE ENCODING
df["Label"] = df["Label"].astype(str).str.strip()  # Convert all labels to string & remove spaces

# ✅ REMOVE INCORRECT NUMERIC STRINGS FROM LABELS
df = df[~df["Label"].str.match(r'^\d+$')]  # Remove labels that are purely numeric strings

# ✅ PRINT UNIQUE LABELS BEFORE ENCODING
print("Unique labels before encoding:", df["Label"].unique())

# ✅ RE-ENCODE LABELS CORRECTLY
label_encoder = LabelEncoder()
df["Label"] = label_encoder.fit_transform(df["Label"])


# ✅ Save LabelEncoder after encoding labels
encoder_path = "/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/label_encoder.pkl"
joblib.dump(label_encoder, encoder_path)

print(f"✅ LabelEncoder saved at: {encoder_path}")


print("✅ Correctly Encoded Labels:", label_encoder.classes_)

Unique labels before encoding: ['DoS Hulk' 'BENIGN' 0 'PortScan' 'DoS Slowhttptest' 'DDoS' 'SSH-Patator'
 'FTP-Patator' 'DoS GoldenEye' 'DoS slowloris' 'Web Attack' 'Bot'
 'Infiltration' 'Heartbleed']
Unique labels before encoding: ['DoS Hulk' 'BENIGN' 'PortScan' 'DoS Slowhttptest' 'DDoS' 'SSH-Patator'
 'FTP-Patator' 'DoS GoldenEye' 'DoS slowloris' 'Web Attack' 'Bot'
 'Infiltration' 'Heartbleed']
✅ LabelEncoder saved at: /content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/label_encoder.pkl
✅ Correctly Encoded Labels: ['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator' 'Web Attack']


# Train-Test Split

In [28]:
# ✅ PREPARE FEATURES AND LABELS
X = df.drop(columns=["Label"])
y = df["Label"]

# ✅ REMOVE CLASSES WITH ONLY ONE INSTANCE
class_counts = Counter(y)
valid_classes = [cls for cls, count in class_counts.items() if count > 1]
df_filtered = df[df["Label"].isin(valid_classes)]

# ✅ SPLIT DATA (STRATIFIED SPLIT TO MAINTAIN CLASS DISTRIBUTION)
X_train, X_test, y_train, y_test = train_test_split(
    df_filtered.drop(columns=["Label"]), df_filtered["Label"],
    test_size=0.30, random_state=42, stratify=df_filtered["Label"]
)

# ✅ CHECK IF ALL CLASSES ARE PRESENT IN BOTH SETS
print("✅ Unique values in y_train:", np.unique(y_train))
print("✅ Unique values in y_test:", np.unique(y_test))


✅ Unique values in y_train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
✅ Unique values in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]


# Apply SMOTE + Undersampling to Balance Classes

In [29]:
# ✅ CHECK TRAINING CLASS DISTRIBUTION
train_class_counts = Counter(y_train)
min_class_size = min(train_class_counts.values())

# ✅ SET `k_neighbors` DYNAMICALLY TO AVOID ERRORS
k_neighbors = min(3, min_class_size - 1) if min_class_size > 1 else 1

# ✅ DEFINE SMOTE STRATEGY (MINIMUM 3000 SAMPLES PER CLASS)
smote_strategy = {cls: max(train_class_counts[cls], 3000) for cls in train_class_counts}
smote = SMOTE(sampling_strategy=smote_strategy, random_state=42, k_neighbors=k_neighbors)

# ✅ APPLY SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("✅ Updated Training Class Distribution:", Counter(y_train_resampled))

✅ Updated Training Class Distribution: Counter({0: 1467647, 4: 120994, 2: 89611, 10: 63573, 3: 7200, 7: 4153, 6: 3770, 5: 3660, 1: 3000, 11: 3000, 12: 3000, 9: 3000, 8: 3000})


# Augment Test Set (If Needed)

In [30]:
# ✅ CHECK TEST CLASS DISTRIBUTION
min_test_samples = 500
test_class_counts = Counter(y_test)
print("Original Test Set Distribution:", test_class_counts)

# ✅ AUGMENT TEST SET IF ANY CLASS HAS < `min_test_samples`
for cls, count in test_class_counts.items():
    if count < min_test_samples and cls in df_filtered["Label"].values:
        additional_samples = df_filtered[df_filtered["Label"] == cls].sample(
            n=min_test_samples - count, replace=True, random_state=42
        )
        X_test = pd.concat([X_test, additional_samples.drop(columns=["Label"])], ignore_index=True)
        y_test = pd.concat([y_test, additional_samples["Label"]], ignore_index=True)

print("✅ Updated Test Set Distribution:", Counter(y_test))

Original Test Set Distribution: Counter({0: 628991, 4: 51855, 2: 38405, 10: 27246, 3: 3086, 7: 1780, 6: 1615, 5: 1568, 11: 966, 12: 643, 1: 586, 9: 11, 8: 3})
✅ Updated Test Set Distribution: Counter({0: 628991, 4: 51855, 2: 38405, 10: 27246, 3: 3086, 7: 1780, 6: 1615, 5: 1568, 11: 966, 12: 643, 1: 586, 9: 500, 8: 500})


In [31]:
from collections import Counter

print("Class distribution in training data (after resampling):")
print(Counter(y_train_resampled))

print("\nClass distribution in testing data:")
print(Counter(y_test))


Class distribution in training data (after resampling):
Counter({0: 1467647, 4: 120994, 2: 89611, 10: 63573, 3: 7200, 7: 4153, 6: 3770, 5: 3660, 1: 3000, 11: 3000, 12: 3000, 9: 3000, 8: 3000})

Class distribution in testing data:
Counter({0: 628991, 4: 51855, 2: 38405, 10: 27246, 3: 3086, 7: 1780, 6: 1615, 5: 1568, 11: 966, 12: 643, 1: 586, 9: 500, 8: 500})


## Save the datasets

In [32]:
# ✅ DEFINE FILE PATHS
train_path = "/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/train.csv"
test_path = "/content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/test.csv"

# ✅ CREATE DATAFRAMES FOR SAVING
train_df = pd.DataFrame(X_train_resampled, columns=X_train.columns)
train_df["Label"] = y_train_resampled

test_df = pd.DataFrame(X_test, columns=X_test.columns)
test_df["Label"] = y_test

# ✅ SAVE TO CSV FILES
train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"✅ Train dataset saved to: {train_path}")
print(f"✅ Test dataset saved to: {test_path}")


✅ Train dataset saved to: /content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/train.csv
✅ Test dataset saved to: /content/drive/MyDrive/Colab Notebooks/XAI IDS Architecture/2_Modeling-Phase/train_and_test_datasets/CICIDS-2017/test.csv
